# Database Example  

In [1]:
# Import libraries
import aopy
from db import dbfunctions as db
from ipywidgets import interactive, widgets
from datetime import date, datetime, timedelta
import re
from tqdm.notebook import tqdm
import numpy as np

In [2]:
# set up constants
preproc_dir = '/data/preprocessed.new'

## Finding a single recording

In [3]:
subject = 'beignet'
task = 'manual control'
project = 'baseline flash'
day = date(2022, 6, 11) # June 11, 2022

entries = db.get_task_entries(subject__name=subject, task__name=task, date=day, project=project)
e = entries[-1]
print(e.project)

baseline flash


In [12]:
# Or use a shortcut
subjects, ids, dates = db.get_mc_sessions(subject=subject, date=day)
print(*zip(subjects, ids, dates))

('beignet', 5624, datetime.date(2022, 6, 11)) ('beignet', 5625, datetime.date(2022, 6, 11))


## Finding a range of dates

In [10]:
start = widgets.DatePicker(
    description='Start date',
    disabled=False
)
end = widgets.DatePicker(
    description='End date',
    disabled=False
)
display(start)
display(end)

DatePicker(value=None, description='Start date')

DatePicker(value=None, description='End date')

In [14]:
# Might want to do some additional filtering by date
subject = 'beignet'
task = 'bmi control'
project = 'linear map'
entries = db.get_task_entries(subject__name=subject, task__name=task, date=(start.value, end.value), project=project)
print(f"{len(entries)} matching recordings")

7 matching recordings


## Select from a dropdown menu

In [15]:
entry_drop = widgets.Dropdown(options=entries)
display(entry_drop)

Dropdown(options=(2022-10-10 13:11:07.272631: beignet on bmi control task, id=6945, 2022-10-10 13:13:48.766118…

In [16]:
e = entry_drop.value
print(e.date)

2022-10-11 09:56:37.170018


## Select anything

In [19]:
start_date = widgets.DatePicker(value = datetime.today() - timedelta(days=30))
end_date = widgets.DatePicker(value = datetime.today())

# Filter by task and subject
entries = db.get_task_entries(date=(start_date.value, end_date.value))
subjects = np.unique([e.subject.name for e in entries])
tasks = np.unique([e.task.name for e in entries])

subj_drop = widgets.Dropdown(options=subjects)
task_drop = widgets.Dropdown(options=tasks) 
entry_select = widgets.SelectMultiple(
    options=[],
    description='entries',
    disabled=False
)

def update_entries(*args):
    entries = db.get_task_entries(
        subj=subj_drop.value, 
        task=task_drop.value,
        date=(start_date.value,end_date.value)
    )
    entry_select.options = ["{} - {}: {}".format(e.id, e.entry_name, e.desc) for i, e in enumerate(entries)]
    
start_date.observe(update_entries)
end_date.observe(update_entries)
subj_drop.observe(update_entries)
task_drop.observe(update_entries)

def select_entry(subject, task, start_date, end_date, entries):
    ids = [int(re.search(r'\d+', e).group()) for e in entries]
    return db.get_task_entries_by_id(ids)


selection = interactive(select_entry,
                     subject=subj_drop,
                     task=task_drop,
                     start_date=start_date,
                     end_date=end_date,
                     entries=entry_select)

display(selection)

/home/aolab/miniconda3/envs/leo-analysis/lib/python3.9/site-packages/traitlets/traitlets.py:697: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


interactive(children=(Dropdown(description='subject', options=('affi', 'beignet', 'test'), value='affi'), Drop…

In [20]:
entries = selection.result
print(f"{len(entries)} selected")

3 selected


## Loading the data

In [4]:
for e in tqdm(entries):
    subject, te_id, date = db.get_entry_details(e)
    
    # Load the data
    exp_data, exp_metadata = aopy.data.load_preproc_exp_data(preproc_dir, subject, te_id, date)

    print(f"entry {te_id} has {len(exp_data['clock'])} clock cycles")

  0%|          | 0/1 [00:00<?, ?it/s]

entry 5624 has 24031 clock cycles
